## Zadanie 1
Wykorzystaj przykłady z notatnika w SQL Windowed Aggregate Functions (cmd 11) i przepisz funkcje używając Spark API.   
Do tego notatnika dopisz użycie funkcji okienkowych LEAD, LAG, FIRST_VALUE, LAST_VALUE, ROW_NUMBER  

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lead, lag, first, last, row_number
from pyspark.sql.window import Window
from datetime import date

#przepisanie z sql
transactions_data = [
    (1, date(2011, 1, 1), 500),
    (1, date(2011, 1, 15), 50),
    (1, date(2011, 1, 22), 250),
    (1, date(2011, 1, 24), 75),
    (1, date(2011, 1, 26), 125),
    (1, date(2011, 1, 28), 175),
    (2, date(2011, 1, 1), 500),
    (2, date(2011, 1, 15), 50),
    (2, date(2011, 1, 22), 25),
    (2, date(2011, 1, 23), 125),
    (2, date(2011, 1, 26), 200),
    (2, date(2011, 1, 29), 250)
]

transactionsDf = spark.createDataFrame(transactions_data, ["AccountId", "TranDate", "TranAmt"])

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import lead, lag, first, last, row_number

#definicja okna (po koncie i sortowanie po dacie)
windowSpec = Window.partitionBy("AccountId").orderBy("TranDate")

#leadto wartość z następnego wiersza
#lag to wartość z poprzedniego
#first to pierwsza transakcja
#last to ostatnia transakcja
#row_number to numeruje wiersze od 1 dla każdej grupy
transactionsDf = transactionsDf.withColumn("NextAmount", lead("TranAmt").over(windowSpec)) \
                               .withColumn("PrevAmount", lag("TranAmt").over(windowSpec)) \
                               .withColumn("FirstAmount", first("TranAmt").over(windowSpec)) \
                               .withColumn("LastAmount", last("TranAmt").over(windowSpec)) \
                               .withColumn("RowNum", row_number().over(windowSpec))

display(transactionsDf)

AccountId,TranDate,TranAmt,NextAmount,PrevAmount,FirstAmount,LastAmount,RowNum
1,2011-01-01,500,50,null,500,500,1
1,2011-01-15,50,250,500,500,50,2
1,2011-01-22,250,75,50,500,250,3
1,2011-01-24,75,125,250,500,75,4
1,2011-01-26,125,175,75,500,125,5
1,2011-01-28,175,null,125,500,175,6
2,2011-01-01,500,50,null,500,500,1
2,2011-01-15,50,25,500,500,50,2
2,2011-01-22,25,125,50,500,25,3
2,2011-01-23,125,200,25,500,125,4
